<a href="https://colab.research.google.com/github/amirsalhuv/MassDetector/blob/main/pytorch_model_YOLOv8Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q ultralytics  comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.7/534.7 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 r

## 1.2 Imports

In [ ]:
# For image manipulation
import cv2
from PIL import Image
import numpy as np
import pandas as pd

# Image loading saving, copy ,measuring time
import time, json, os
import datetime
import pickle
# For plots
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
import random

# Pytorch
import torch
from torch.utils.data import DataLoader,Dataset
from torch.nn import BCELoss
from torchvision.transforms import Compose, ToTensor, Normalize
from torch import nn,optim
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torchvision.models import vgg16_bn, VGG16_BN_Weights
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# 1.3 Definitions


In [ ]:
# Amir Salhuv:
dataset_dir = '/content/drive/MyDrive/Project_OpenU/CBIS-DDSM/manifest/'

# Yonatan Salhuv:
#dataset_dir = '/content/drive/MyDrive/'

training_folder = dataset_dir + "Sparse_data_for_model/"
run_mode = "evaluate_model"

# Yolo training
yolo_base_folder = '/content/drive/MyDrive/Project_OpenU/CBIS-DDSM/Latest_model_data/'
yolo_training_foder = yolo_base_folder + 'Images/train'
yolo_validation_foder = yolo_base_folder + 'Images/val'


# Classes and batch size
n_classes=2
batch_size = 16

# Split to train and validation folder

In [ ]:
if run_mode == "create_yolo_train_test_split": # todo - needs update for Yolo folders shape (done manually)
  # directories
  img_dir = training_folder+'Images_new/'
  mask_dir = training_folder+'Masks_new/'

  # Images
  images_path = sorted(os.listdir(img_dir))
  masks_path = sorted(os.listdir(mask_dir))

  # Assuming X is your data and y are your labels.
  X_train, X_test, y_train, y_test = train_test_split(images_path, masks_path, test_size=0.2, random_state=42)

  for filename in tqdm(images_path):
    # If in training set, put in the training folder
    if filename in X_train:
      shutil.copy(img_dir + filename, yolo_training_foder +'Images/' + filename)
      shutil.copy(mask_dir + filename, yolo_training_foder + 'Masks/' + filename)
    else:
      shutil.copy(img_dir + filename, yolo_validation_foder +'Images/' + filename)
      shutil.copy(mask_dir + filename, yolo_validation_foder + 'Masks/' + filename)

## Create the Yolo folder

In [ ]:
if run_mode == "create_yolo_folder": # todo - needs update for Yolo folders shape (done manually)

  # directories:
  # Training
  training_img_dir = yolo_training_foder +'Images'
  training_mask_dir = yolo_training_foder +'Masks'
  training_output_dir = yolo_training_foder +'Labels'
  # ensure output directory exists
  os.makedirs(training_output_dir, exist_ok=True)

  # Validation
  validation_img_dir = yolo_validation_foder +'Images'
  validation_mask_dir = yolo_validation_foder +'Masks'
  validation_output_dir = yolo_validation_foder +'Labels'
  # ensure output directory exists
  os.makedirs(validation_output_dir, exist_ok=True)



  def convert_to_yolo_format(mask_image,mask_filename):
      # find contours in the mask
      contours, _ = cv2.findContours(mask_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
      # Filter contours based on area
      min_contour_area = 100  # Minimum area to be considered a valid contour
      contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
      if len(contours)>1:
        print(f"\nmask bigger than 1: {mask_filename}, number of contours {len(contours)}")
      elif len(contours)==0:
        print(f"\nMask error{mask_filename}")

      yolo_annotations = []
      for contour in contours:
          # compute bounding box
          x, y, w, h = cv2.boundingRect(contour)
          # convert to yolo format
          x_center = (x + w / 2) / mask_image.shape[1]
          y_center = (y + h / 2) / mask_image.shape[0]
          width = w / mask_image.shape[1]
          height = h / mask_image.shape[0]
          # object class is 0 as we assume we only have one class
          yolo_annotations.append(f'0 {x_center} {y_center} {width} {height}')

      return yolo_annotations

    # Training folder
  for filename in tqdm(sorted(os.listdir(training_img_dir))):
      # read the image and mask
      mask = cv2.imread(os.path.join(training_mask_dir, filename), cv2.IMREAD_GRAYSCALE)* 255

      # convert mask to yolo format
      yolo_annotations = convert_to_yolo_format(mask,filename)

      # write annotations to file
      with open(os.path.join(training_output_dir, filename.replace('.png', '.txt')), 'w') as f:
          for annotation in yolo_annotations:
              f.write(annotation + '\n')

  # Validation folder
  for filename in tqdm(sorted(os.listdir(validation_img_dir))):
      # read the image and mask
      mask = cv2.imread(os.path.join(validation_mask_dir, filename), cv2.IMREAD_GRAYSCALE)* 255

      # convert mask to yolo format
      yolo_annotations = convert_to_yolo_format(mask,filename)

      # write annotations to file
      with open(os.path.join(validation_output_dir, filename.replace('.png', '.txt')), 'w') as f:
          for annotation in yolo_annotations:
              f.write(annotation + '\n')


# 5 - Training the model

In [ ]:
if run_mode=="training_model" or run_mode=="evaluate_model":
  # Clone YOLOv5 repository
  !git clone -q https://github.com/ultralytics/yolov5.git

  # for running the train.py from the yolov5 directory
  %cd yolov5

  # Install dependencies
  !pip install -q -r requirements.txt

/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
if run_mode=="training_model":
  # This is an example data configuration. Replace with your own.
  # Assume that this file is stored in the current directory.
  train_dir = 'images/train'
  val_dir = 'images/val'
  num_classes = 1
  class_names = ['mass']  # Replace with your actual class names

  data_yaml = """
  path: {} # dataset root dir
  train: {}
  val: {}
  nc: {}  # number of classes
  names: {}  # class names
  """.format(yolo_base_folder,train_dir, val_dir, num_classes, class_names)

  with open("data.yaml", "w") as file:
      file.write(data_yaml)

  !python train.py --img 640 --batch 16 --epochs 100 --data data.yaml --cfg yolov5x.yaml --weights yolov5x.pt --project training_folder+"yolov5_models"


## 5.1 Define the Unet block

### 5.2.1 Option 1 - iou loss

In [ ]:
if run_mode == 'training_model':

  def iou_loss(pred, target):
      smooth = 1.0
      print(f'shapes: pred {pred.shape} target {target.shape}')
      iflat = pred.view(-1)
      tflat = target.view(-1)
      intersection = (iflat * tflat).sum()
      return 1 - ((intersection + smooth) / (iflat.sum() + tflat.sum() - intersection + smooth))

# 5.4 Evaluate model

In [ ]:
if run_mode == 'evaluate_model':

  iou = []
  epsilon = 10e-6
  # Load model
  path_or_model = training_folder + 'yolov5_models/exp2/weights/best.pt'
  model = torch.hub.load('ultralytics/yolov5', 'custom', path=path_or_model)

  # mask path
  mask_base_path = yolo_base_folder + '/masks/Validation/Masks_val/'

    # If using a GPU
  if torch.cuda.is_available():
    model = model.cuda()

  model.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-6-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/requests-2.27.1.dist-info/METADATA'


Fusing layers... 
YOLOv5x summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


In [ ]:
if run_mode == 'evaluate_model':

  val_image_dir = yolo_base_folder +'images/val'
  val_list = os.listdir(val_image_dir)
  # Steps:
  # 1. take an image
  # 2. inference from image
  # 3. draw a masks from the results
  # 5. Compute the IOU
  # 6. if iou >10% consider as good detection
  for i,image_path in enumerate(val_list):

    # step #1 - read the image and mask
    img = cv2.imread(os.path.join(val_image_dir, image_path))
    mask = cv2.imread(mask_base_path + image_path,cv2.IMREAD_GRAYSCALE)* 255
    predicted_rect= np.zeros_like(mask)

    # Step #2 -inference from image
    results = model(os.path.join(val_image_dir, image_path))

    # step #3- draw masks from result
    for *box, _, _ in results.pred[0]:
      # Convert box format from [center_x, center_y, width, height] to [x1, y1, x2, y2]
      x, y, box_width, box_height = box
      x1 = int(x)
      y1 = int(y)
      x2 = int(box_width)
      y2 = int(box_height)

      # Draw rectangle (full)
      cv2.rectangle(predicted_rect, (x1, y1), (x2, y2), (255, 255, 255), -1)

    # Step #4 - IOU of mask and image
    # first - convert to shape of w,h
    #predicted_rect = cv2.cvtColor(predicted_rect,cv2.COLOR_BGR2GRAY)
    intersection = np.sum(predicted_rect * mask)
    union = np.sum(predicted_rect)+np.sum(mask)

    iou_temp = intersection/(union+epsilon)

    # Summary of the results - add the IOU results list
    iou.append(iou_temp*1000)
    #print(f" iou: {iou[i]*1000}, intesection: {intersection}, union : {union}")

In [ ]:
 if run_mode == 'evaluate_model':
    total = np.sum(np.array(iou)>0)/len(iou)*100
    misses = len(iou)-np.sum(np.array(iou)>0)
    print(f"number of misdetection: {misses}, misdetection rate: {100-total:.2f}%")


number of misdetection: 19, misdetection rate: 9.09%


In [ ]:
 if run_mode == 'evaluate_model':

  import cv2
  import matplotlib.pyplot as plt
  #%matplotlib inline

  # Load image
  image_path = yolo_base_folder + 'images/val/1042.png'
  img = cv2.imread(image_path)

  # Get image dimensions
  height, width = img.shape[:2]

  # Run inference
  results = model(image_path)

  # Class labels (replace with your actual labels)
  class_labels = ['mass']

  # Loop over each prediction
  for *box, confidence, class_id in results.pred[0]:
      # Convert box format from [center_x, center_y, width, height] to [x1, y1, x2, y2]
      x, y, box_width, box_height = box
      print(box)
      x1 = int(x)
      y1 = int(y)
      x2 = int(box_width)
      y2 = int(box_height)

      # Get class label
      class_label = class_labels[int(class_id)]

      # Draw rectangle and label on image
      cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
      cv2.putText(img, f'{class_label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

  # Create a figure and axis
  fig, ax = plt.subplots(1,figsize=(10,10))

  # Display the image
  ax.imshow(img)
  # Show the figure with the bounding boxes
  plt.show()

[tensor(55.89012, device='cuda:0'), tensor(3018.40063, device='cuda:0'), tensor(435.42810, device='cuda:0'), tensor(3375.81348, device='cuda:0')]


In [ ]:
print(results.pred)

[tensor([[5.58901e+01, 3.01840e+03, 4.35428e+02, 3.37581e+03, 6.61184e-01, 0.00000e+00]], device='cuda:0')]
